# Simulation Control Notebook

This notebook will handle all of the scripting that would typically be done in bash but I decided to try in python to keep in with the primary theme
This should also help a TON with the math and such since bash doesn't do decimal arithmetic by itself.

In [1]:
from subprocess import run

## Actual Simulation Portion

The depth does simulation is a pretty easy simulation to comprehend. The python here is just to automate the switching of materials between `Water` and `ScintX`. It also changes the output filename to convey which material was used. The module `sys` is being utilized, `sys.run`. `sys.run` allows a command from the terminal Ex. `ls`, `cd`, `sed`, and `grep` to be issued in another process. This allows me to use the math and logic of Python to cleanly impliment the looping then use the terminal's `sed` command to change a pattern of text in the ex1.mac file to what I need it to be.

The water or scintx simulations will be controlled by a single `for` loop that loops through the strings `'Water'` and `'ScintX'`. They will get inserted into the paths of the output depth files.

In [2]:
number_of_primaries = 1e7 # This will get turned into an integer when inserted into the file

In [3]:
for energy in ['6', '10']:
    for material in ['Water', 'ScintX']:

        # Changes material from Water to ScintX along with the output file names
        run(f'sed -i "s:/gate/actor/depthdose/save .*:/gate/actor/depthdose/save output/E{energy.zfill(2)}_{material}_depth_dose.txt:" mac/ex1.mac', shell=True)
        run(f'sed -i "s:/gate/waterbox/setMaterial.*:/gate/waterbox/setMaterial {material}:" mac/ex1.mac', shell=True)
        run(f'sed -i "s:/gate/application/setTotalNumberOfPrimaries.*:/gate/application/setTotalNumberOfPrimaries {int(number_of_primaries)}:" mac/ex1.mac', shell=True)
        run(f'sed -i "s:/gate/source/mybeam/gps/ene/mono.*:/gate/source/mybeam/gps/ene/mono {energy} MeV:" mac/ex1.mac', shell=True)
        run(f'sed -i "s:/gate/actor/doseprofile/save.*:/gate/actor/doseprofile/save output/E{energy.zfill(2)}_{material}_profile_dose.txt:" mac/ex1.mac', shell=True)


        run('Gate mac/ex1.mac', shell=True)

### Sed
`sed` is a tool developed for text manipulation of files. It is **extremely** fast and light weight. It is also a product of the early days of computing so following with sterotypes it is 'clunky and fragile' to use. I will walk you through what is going on.

- First:
`sed` stands for "stream editor" which makes sense since all it does is edit sequences of text in streams. `sed -i` tells `sed` to do its thing **Inline**. Normally it would "open" the file to memory, do its edits on the file in memory then print it out to standard out or the user would redirect the printing to another file. This is useful but not what we want, we want the file to be edited right then and there.

- Second
`s:` stands for substitution, `sed` matches patterns of text based on **regular expressions**. The pattern following `:` is the pattern being looked for, the `.*` matches any amount of any character its a super wild card. The stuff following the second `:` is what the first pattern will be replaced or substituted with.
- This is where Python makes this easy, I'm using an fstring to create a string and put the value of the variable width divided by 2 and with 2 decimals in the location it appears in the string. I get easy formatting and string manipulation unlike bash
The final `:` denotes the end of the `sed` comand and the `"` maks the end of the pattern stuff to the command. The patterns must be passed to `sed` as strings which is why I am using `"`.

- Third
The last part of the string `mac/ex1.mac` is the file being changed by `sed`